In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.5 MB/s eta 0:00:00


In [24]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [21]:
data=pd.read_csv('/content/drive/MyDrive/world_marathon_majors.csv', encoding='cp1252')
data.tail()

,year,winner,gender,country,time,marathon
536,1978,Mark Stanforth,Male,United States,2018-05-04 02:19:20,Chicago
537,1978,Lynae Larson,Female,United States,2018-05-04 02:59:25,Chicago
538,1977,Dan Cloeter,Male,United States,2018-05-04 02:17:52,Chicago
539,1977,Dorothy Doolittle,Female,United States,2018-05-04 02:50:47,Chicago
540,2018,Vivian Cheruiyot,Female,Kenya,2018-05-04 02:18:31,London


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      541 non-null    int64 
 1   winner    536 non-null    object
 2   gender    536 non-null    object
 3   country   536 non-null    object
 4   time      536 non-null    object
 5   marathon  541 non-null    object
dtypes: int64(1), object(5)
memory usage: 25.5+ KB


In [5]:
data.columns

Index(['year', 'winner', 'gender', 'country', 'time', 'marathon'], dtype='object')

In [6]:
from datetime import datetime as dt

In [7]:
# Assuming 'data' is your DataFrame and 'time' is the column containing your datetime strings
data['time'] = pd.to_datetime(data['time'], format="%Y-%m-%d %H:%M:%S")

# Extract hours and minutes
data['hours'] = data['time'].dt.hour
data['minutes'] = data['time'].dt.minute

# Convert to seconds
data['finish_time_seconds'] = (data['hours'] * 3600) + (data['minutes'] * 60)

print(data['finish_time_seconds'])

0       7500.0
1       8340.0
2       7380.0
3       8340.0
4       7560.0
        ...   
536     8340.0
537    10740.0
538     8220.0
539    10200.0
540     8280.0
Name: finish_time_seconds, Length: 541, dtype: float64


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   year                 541 non-null    int64         
 1   winner               536 non-null    object        
 2   gender               536 non-null    object        
 3   country              536 non-null    object        
 4   time                 536 non-null    datetime64[ns]
 5   marathon             541 non-null    object        
 6   hours                536 non-null    float64       
 7   minutes              536 non-null    float64       
 8   finish_time_seconds  536 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 38.2+ KB


In [9]:
winner_df=data.copy().drop(columns=['time', 'hours', 'minutes','winner'])
winner_df.head()

,year,gender,country,marathon,finish_time_seconds
0,2018,Male,Kenya,Tokyo,7500.0
1,2018,Female,Ethiopia,Tokyo,8340.0
2,2017,Male,Kenya,Tokyo,7380.0
3,2017,Female,Kenya,Tokyo,8340.0
4,2016,Male,Ethiopia,Tokyo,7560.0


In [10]:
gender_dummies=pd.get_dummies(winner_df['gender'], drop_first=True, dtype=int)
dummies = pd.get_dummies(winner_df, columns=['country', 'marathon'], dtype=int)


In [47]:
combined_df = pd.concat([ dummies, gender_dummies], axis=1)
combined_df.drop(columns=['gender'], inplace=True)
combined_df

,year,finish_time_seconds,country_Australia,country_Belgium,country_Brazil,country_Canada,country_China,country_Colombia,country_Denmark,country_Eritrea,...,country_United Kingdom,country_United States,country_Yugoslavia,marathon_Berlin,marathon_Boston,marathon_Chicago,marathon_London,marathon_NYC,marathon_Tokyo,Male
0,2018,7500.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,2018,8340.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2017,7380.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,2017,8340.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2016,7560.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,1978,8340.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
537,1978,10740.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
538,1977,8220.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
539,1977,10200.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [16]:
combined_df.columns

Index(['year', 'finish_time_seconds', 'country_Australia', 'country_Belgium',
       'country_Brazil', 'country_Canada', 'country_China', 'country_Colombia',
       'country_Denmark', 'country_Eritrea', 'country_Ethiopia',
       'country_Finland', 'country_Germany', 'country_Greece',
       'country_Guatemala', 'country_Hungary', 'country_Ireland',
       'country_Italy', 'country_Japan', 'country_Kenya', 'country_Latvia',
       'country_Mexico', 'country_Morocco', 'country_New Zealand',
       'country_Norway', 'country_Poland', 'country_Portugal',
       'country_Romania', 'country_Russia', 'country_South Africa',
       'country_South Korea', 'country_Soviet Union', 'country_Spain',
       'country_Sweden', 'country_Switzerland', 'country_Tanzania',
       'country_United Kingdom', 'country_United States', 'country_Yugoslavia',
       'marathon_Berlin', 'marathon_Boston', 'marathon_Chicago',
       'marathon_London', 'marathon_NYC', 'marathon_Tokyo', 'Male'],
      dtype='object')

In [17]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    541 non-null    int64  
 1   finish_time_seconds     536 non-null    float64
 2   country_Australia       541 non-null    int64  
 3   country_Belgium         541 non-null    int64  
 4   country_Brazil          541 non-null    int64  
 5   country_Canada          541 non-null    int64  
 6   country_China           541 non-null    int64  
 7   country_Colombia        541 non-null    int64  
 8   country_Denmark         541 non-null    int64  
 9   country_Eritrea         541 non-null    int64  
 10  country_Ethiopia        541 non-null    int64  
 11  country_Finland         541 non-null    int64  
 12  country_Germany         541 non-null    int64  
 13  country_Greece          541 non-null    int64  
 14  country_Guatemala       541 non-null    in

In [26]:
cleaned_df=combined_df.dropna()

In [29]:
cleaned_df

,year,finish_time_seconds,country_Australia,country_Belgium,country_Brazil,country_Canada,country_China,country_Colombia,country_Denmark,country_Eritrea,...,country_United Kingdom,country_United States,country_Yugoslavia,marathon_Berlin,marathon_Boston,marathon_Chicago,marathon_London,marathon_NYC,marathon_Tokyo,Male
0,2018,7500.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,2018,8340.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2017,7380.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,2017,8340.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2016,7560.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,1978,8340.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
537,1978,10740.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
538,1977,8220.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
539,1977,10200.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [30]:
y=cleaned_df['finish_time_seconds']
X=cleaned_df.drop(columns=['finish_time_seconds'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Define the deep learning model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=45)) # Changed input_dim to 45
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: -3149.4092
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -14505.0889 
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -38041.7734 
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -90558.4766 
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -214761.6250 
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: -486861.4375 
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -1046159.7500
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -2113117.0000 
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -4062807.7500 
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: -7506663.5000 
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 

In [42]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=3)

In [43]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [44]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [46]:
# Print the classification report comparing the testing data to the model predictions
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      7320.0       0.00      0.00      0.00         1
      7380.0       0.00      0.00      0.00         3
      7440.0       0.00      0.00      0.00         1
      7500.0       0.00      0.00      0.00         1
      7560.0       0.17      1.00      0.29         1
      7620.0       0.12      0.20      0.15         5
      7680.0       0.00      0.00      0.00         6
      7740.0       0.18      0.29      0.22         7
      7800.0       0.00      0.00      0.00         5
      7860.0       0.00      0.00      0.00         5
      7920.0       0.00      0.00      0.00         2
      7980.0       0.00      0.00      0.00         1
      8040.0       0.00      0.00      0.00         5
      8100.0       0.00      0.00      0.00         4
      8160.0       0.00      0.00      0.00         2
      8220.0       0.00      0.00      0.00         1
      8280.0       0.12      0.25      0.17         4
      8340.0       0.33    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_